# Claude vs GPT

**Improvements**
- Pause the convo and continue without restarting.
- Choose the topic ✅

In [ ]:
import sys
sys.path.append("../src")

from chat.chatbot import Socrates, Eris
from chat.test_chatbot import TestEris, TestSocrates
from streaming.stream import stream_with_typing_and_audio
# from audio.test_speak import speak_message

import gradio as gr
import random

In [ ]:
# !apt-get -qq -y install espeak-ng > /dev/null 2>&1
# !winget install ffmpeg        # Required with pydub

## Debate: Claude VS GPT

### Main Function

In [ ]:
def adversarial_chat(n_rounds, topic, speed=0.03, voice="kokoro"):
    socrates, eris = TestSocrates(), TestEris()
    history = []

    # --- opening taunt from Eris ---
    eris_reply = random.choice([
        f"Ready to get schooled about '{topic}', old man?",
        f"Today's topic is '{topic}'. This should be easy—your logic is about as sharp as a wet napkin.",
        f"Hope you're prepared to lose spectacularly, Socrates. Topic: '{topic}'.",
        f"Hey Socrates, wanna argue about '{topic}'? Enjoy losing.",
    ])
    generator = stream_with_typing_and_audio(
        history, 
        eris_reply,
        is_socrates=False,
        delay=speed,
        voice=voice
    )
    for history, audio in generator:
        yield history, audio

    # --- remaining rounds ---
    for turn in range(n_rounds - 1):
        # Socrates turn -> Even turns
        if turn % 2 == 0:                   
            socrates.history = eris.history.copy()
            soc_reply = socrates.stream(eris_reply)
            generator = stream_with_typing_and_audio(
                history, soc_reply,
                is_socrates=True,
                delay=speed,
                voice=voice
            )
            for history, audio in generator:
                yield history, audio
        # Eris' turn -> Odd turns
        else:                               
            eris.history = socrates.history.copy()
            eris_reply = eris.stream(soc_reply)
            generator = stream_with_typing_and_audio(
                history, eris_reply,
                is_socrates=False,
                delay=speed,
                voice=voice
            )
            for history, audio in generator:
                yield history, audio


### GUI

In [7]:
with gr.Blocks() as demo:
    gr.Markdown("## ⚔️ Socrates vs. Eris")
    gr.Markdown(
        "Dive into a battle of minds where **Socrates**, the calm philosopher, asks piercing questions "
        "to uncover truth — and **Eris**, the goddess of discord, responds with sarcastic wit and playful contradiction.  \n"
        "Choose a topic and watch them clash in a brief, animated debate."
    )

    with gr.Row():
        rounds = gr.Number(label="Rounds", value=4, precision=0, minimum=1)
        topic_input = gr.Textbox(label="Debate Topic", placeholder="e.g. What is freedom?")
        start_btn = gr.Button("Start Debate", interactive=True) #False)  # Initially disabled

    chatbox = gr.Chatbot(label="Let the battle begin!", type="messages", height=500)
    audio_output = gr.Audio(label="🔊 Speech", autoplay=True)

    def reset():
        return [], None  

    def validate_inputs(topic, n_rounds):
        valid_topic = bool(topic.strip())
        valid_rounds = isinstance(n_rounds, (int, float)) and n_rounds > 0
        return gr.update(interactive=valid_topic and valid_rounds)

    start_btn.click(fn=reset, outputs=[chatbox, audio_output])
    start_btn.click(
        fn=adversarial_chat, 
        inputs=[rounds, topic_input],
        outputs=[chatbox, audio_output]
    )

    # Enable/disable start button based on both inputs
    topic_input.change(fn=validate_inputs, inputs=[topic_input, rounds], outputs=start_btn)
    rounds.change(fn=validate_inputs, inputs=[topic_input, rounds], outputs=start_btn)


demo.launch(share=False)

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


stream_with_typing_and_audio -> history: Today's topic is 'j'. This should be easy—your logic is about as sharp as a wet napkin.


c:\Users\becaye.balde\Desktop\GitHub\LLMs\Advserial-LLMs\.venv\Lib\site-packages\gradio\processing_utils.py:753: UserWarning: Trying to convert audio automatically from float32 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))


stream_with_typing_and_audio -> history: <generator object TestSocrates.stream at 0x0000027C58E88400>


Traceback (most recent call last):
  File "c:\Users\becaye.balde\Desktop\GitHub\LLMs\Advserial-LLMs\.venv\Lib\site-packages\gradio\queueing.py", line 715, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\becaye.balde\Desktop\GitHub\LLMs\Advserial-LLMs\.venv\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\becaye.balde\Desktop\GitHub\LLMs\Advserial-LLMs\.venv\Lib\site-packages\gradio\blocks.py", line 2191, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\becaye.balde\Desktop\GitHub\LLMs\Advserial-LLMs\.venv\Lib\site-packages\gradio\blocks.py", line 1714, in call_function
    prediction = await utils.async_iteration(iterator)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\becaye.ba